# Ollama Introduction
This little notebook is a brief introduction to Ollama, a tool for interacting with open LLMs deployed on Ollama server (the server can also run locally).

In [1]:
from ollama import Client

# initialize the Ollama client with the specified host
ollama_host = "http://localhost:11434"
client = Client(host=ollama_host)

In [2]:
# Get a list of all models that are currently downloaded
models = client.list()
print(models.models)

MODEL_NAME = "gemma3"

[Model(model='gemma3:latest', modified_at=datetime.datetime(2025, 6, 11, 16, 51, 6, 220107, tzinfo=TzInfo(+02:00)), digest='a2af6cc3eb7fa8be8504abaf9b04e88f17a119ec3f04a3addf55f92841195f5a', size=3338801804, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='4.3B', quantization_level='Q4_K_M')), Model(model='gemma3:27b', modified_at=datetime.datetime(2025, 6, 11, 16, 45, 59, 91280, tzinfo=TzInfo(+02:00)), digest='a418f5838eaf7fe2cfe0a3046c8384b68ba43a4435542c942f9db00a5f342203', size=17396936941, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='27.4B', quantization_level='Q4_K_M'))]


In [3]:
import json
import os
from pydantic import BaseModel
from typing import List

class QuestionResponse(BaseModel):
    questions: List[str]

def get_questions_from_review(review: str) -> List[str]:
    """
    Extracts scientific questions from a peer review.
    """
    response = client.chat(
        model=MODEL_NAME,
        messages=[
            {"role": "user", 
             "content": (
                    "You are given a peer review of a scientific article.\n\n"
                    "Your task is to identify and extract all **scientific, information-seeking questions** posed by the reviewer.\n\n"
                    "Only include questions related to the **scientific content** of the article — such as methodology, experimental design, data analysis, results, interpretation, assumptions, or scientific relevance.\n\n"
                    "**Exclude** any questions about grammar, spelling, formatting, or writing style.\n\n"
                    "If necessary, **rephrase the questions** so they are self-contained and independent of the original review context.\n\n"
                    "Return the extracted questions as a list. \n\n"
                    f"Peer review:\n{review}"
            )}
        ],
        options={
            "temperature": 0.8,
            "num_predict": 1024,
        },
        format=QuestionResponse.model_json_schema()
    )
    structured_response = QuestionResponse.model_validate_json(response.message.content)
    return structured_response.questions

base_dir = 'test-set'

for dir in os.listdir(base_dir)[2:]:
    dir_path = os.path.join(base_dir, dir)
    if os.path.isdir(dir_path):
        print(f"Processing directory: {dir_path}")

        questions_json = os.path.join(dir_path, 'questions_from_reviews.json')
        if os.path.exists(questions_json):
            print(f"Questions file already exists: {questions_json}")
            continue

        # Load the peer reviews from a JSON file
        review_file = os.path.join(dir_path, 'all_reviews.json')

        with open(review_file, 'r', encoding='utf-8') as file:
            reviews = json.load(file)
        
        result = []
        # Process each review
        for review in reviews:
            questions = get_questions_from_review(review)
            result.extend(questions)

        # Save the results 
        with open(questions_json, 'w', encoding='utf-8') as file:
            json.dump(result, file, ensure_ascii=False, indent=4)
        print(f"Questions saved to {questions_json}")

Processing directory: test-set\gmd-18-3311-2025
Questions file already exists: test-set\gmd-18-3311-2025\questions_from_reviews.json
Processing directory: test-set\hgss-15-71-2024
Questions file already exists: test-set\hgss-15-71-2024\questions_from_reviews.json
Processing directory: test-set\mr-6-119-2025
Questions file already exists: test-set\mr-6-119-2025\questions_from_reviews.json
Processing directory: test-set\s42004-025-01575-2
Questions saved to test-set\s42004-025-01575-2\questions_from_reviews.json
Processing directory: test-set\s43247-025-02414-x
Questions saved to test-set\s43247-025-02414-x\questions_from_reviews.json


## Example

In [ ]:
# Send a simple prompt to the model
# model: select a model from the list of models obtained from client.list()
# messages: a list of messages containing the conversation history. Some models also 
# have a system message, to add this, make the first message:
# {"role": "system", "content": "Your system message here"}
# Gemma3 does not have a system message, so we can start with the user message.
# To add responses from the model, you can use the "assistant" role, i.e.:
# {"role": "assistant", "content": "The capital of France is Paris."}

response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
        {"role": "assistant", "content": "The capital of France is Paris."},
        {"role": "user", "content": "And who many people live there?"},
    ],
)
print(response.message.content)

As of 2023, the population of Paris is approximately **2.1 million** people.

It’s important to note that this refers to the city proper (the administrative limits). The Greater Paris metropolitan area, which includes surrounding suburbs, has a much larger population – over 11 million! 

Would you like to know more about the population of Paris or its surrounding area?


In [7]:
# To control the decoding parameters, such as temperature, maxium number of tokens, etc.,
# you can pass additional parameters to the chat method.
# For a complete list of options, check the Ollama API documentation at:
# https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values
response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
        {"role": "assistant", "content": "The capital of France is Paris."},
        {"role": "user", "content": "And who many people live there?"},
    ],
    options={
        "temperature": 0.7,
        "num_predict": 1024,
    }
)
print(response.message.content)

As of 2023, the population of Paris is approximately **2.1 million** people within the city limits. However, the greater Paris metropolitan area (which includes surrounding suburbs) has a population of over **11 million**! 

Would you like to know anything more about Paris?


In [8]:
# To force the model to generate a structured response, i.e. a JSON object,
# you can define a schema for the response and pass it. The schema can be defined using 
# Pydantic models and the built-in python types (more complex types are also supported, 
# check the pydantic documentation for more details).

from pydantic import BaseModel
class PopulationResponse(BaseModel):
    city: str
    population: int

response = client.chat(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": "What is the capital of France?"},
        {"role": "assistant", "content": "The capital of France is Paris."},
        {"role": "user", "content": "And who many people live there?"},
    ],
    format=PopulationResponse.model_json_schema()
)
structured_response = PopulationResponse.model_validate_json(response.message.content)
print(structured_response)

city='Paris' population=2141636
